This notebook provides a simple sudoku solver. Let's start with a simple 4 (\*\*\*\*) star sudoku

In [1]:
# https://www.adrian.idv.hk/2019-01-30-simanneal/
# Parool Dinsdag 19 sept ****
sudoku_grid = [
    [0, 0, 0, 0, 0, 0, 0, 9, 2],
    [0, 0, 0, 1, 0, 0, 0, 4, 0],
    [9, 0, 0, 2, 4, 0, 0, 0, 7],
    [8, 0, 0, 7, 0, 0, 1, 5, 0],
    [6, 5, 0, 9, 0, 1, 0, 7, 8],
    [0, 7, 4, 0, 0, 8, 0, 0, 6],
    [3, 0, 0, 0, 9, 5, 0, 0, 1],
    [0, 8, 0, 0, 0, 6, 0, 0, 0],
    [7, 9, 0, 0, 0, 0, 0, 0, 0]
]


problem = [x for row in sudoku_grid for x in row]

In [2]:
def coord(row, col):
    return row*9+col

def block_indices(block_num):
    """return linear array indices corresp to the sq block, row major, 0-indexed.
    block:
       0 1 2     (0,0) (0,3) (0,6)
       3 4 5 --> (3,0) (3,3) (3,6)
       6 7 8     (6,0) (6,3) (6,6)
    """
    firstrow = (block_num // 3) * 3
    firstcol = (block_num % 3) * 3
    indices = [coord(firstrow+i, firstcol+j) for i in range(3) for j in range(3)]
    return indices


block_indices(1)

[3, 4, 5, 12, 13, 14, 21, 22, 23]

In [3]:
import random

def initial_solution(problem):
    """provide sudoku problem, generate an init solution by randomly filling
    each sq block without considering row/col consistency"""
    solution = problem.copy()
    for block in range(9):
        indices = block_indices(block)
#        block = problem[indices]
        block = [problem[i] for i in indices]
        zeros = [i for i in indices if problem[i] == 0]
        to_fill = [i for i in range(1, 10) if i not in block]
        random.shuffle(to_fill)
        for index, value in zip(zeros, to_fill):
            solution[index] = value
    return solution


initial_solution(problem)

[3,
 7,
 4,
 7,
 6,
 3,
 6,
 9,
 2,
 8,
 5,
 2,
 1,
 8,
 5,
 5,
 4,
 1,
 9,
 1,
 6,
 2,
 4,
 9,
 3,
 8,
 7,
 8,
 2,
 1,
 7,
 3,
 4,
 1,
 5,
 2,
 6,
 5,
 3,
 9,
 2,
 1,
 4,
 7,
 8,
 9,
 7,
 4,
 5,
 6,
 8,
 9,
 3,
 6,
 3,
 2,
 4,
 8,
 9,
 5,
 3,
 8,
 1,
 1,
 8,
 5,
 7,
 1,
 6,
 5,
 2,
 7,
 7,
 9,
 6,
 4,
 3,
 2,
 4,
 9,
 6]

In [4]:
from simanneal import Annealer

class Sudoku_Sq(Annealer):
    def __init__(self, problem):
        self.problem = problem
        state = initial_solution(problem)
        super().__init__(state)
    def move(self):
        """randomly swap two cells in a random square"""
        block = random.randrange(9)
        indices = [i for i in block_indices(block) if self.problem[i] == 0]
        m, n = random.sample(indices, 2)
        self.state[m], self.state[n] = self.state[n], self.state[m]
    def energy(self):
        """calculate the number of violations: assume all rows are OK"""
        column_score = lambda n: -len(set(self.state[coord(i, n)] for i in range(9)))
        row_score = lambda n: -len(set(self.state[coord(n, i)] for i in range(9)))
        score = sum(column_score(n)+row_score(n) for n in range(9))
        if score == -162:
            self.user_exit = True # early quit, we found a solution
        return score


In [8]:
sudoku = Sudoku_Sq(problem)
sudoku.copy_strategy = "method"

sudoku.Tmax = 0.5
sudoku.Tmin = 0.05
sudoku.steps = 100000
sudoku.updates = 100
state, e = sudoku.anneal()
print("\n")

print("E=%f (expect -162)" % e)


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.22854       -160.00     1.80%     0.10%     0:00:01     0:00:01



E=-162.000000 (expect -162)


In [9]:
from sudoku.printer import display_list

display_list(state)

4 1 8 | 6 5 7 | 3 9 2 
2 3 7 | 1 8 9 | 6 4 5 
9 6 5 | 2 4 3 | 8 1 7 
------+-------+------
8 2 9 | 7 6 4 | 1 5 3 
6 5 3 | 9 2 1 | 4 7 8 
1 7 4 | 5 3 8 | 9 2 6 
------+-------+------
3 4 2 | 8 9 5 | 7 6 1 
5 8 1 | 4 7 6 | 2 3 9 
7 9 6 | 3 1 2 | 5 8 4 
